In [1]:
import numpy as np
p = np.array([0.6, 0.4])
sigma = np.array([[0.0287, 0], [0, 0.0017]])
p_var = p @ sigma @ p
risk_contribs = np.diag(p) @ sigma @ p / (p @ sigma @ p)
risk_contribs

array([0.9743493, 0.0256507])

In [2]:
lamb = 4.0
gamma = 0.3
sigma = np.array([[0.0287, 0], [0, 0.0017]])
mu = gamma * np.sqrt(np.diag(sigma))
 
w_star = 1.0 / lamb * np.matrix(sigma).I @ mu
w_star = np.asarray(w_star).flatten()
 
w_var = w_star @ sigma @ w_star     
risk_contribs = np.diag(w_star) @ sigma @ w_star / w_var
risk_contribs

array([0.5, 0.5])